In [1]:
import tensorflow as tf
import pickle
import numpy as np
import timeit
import time
from load_cifar import *

In [ ]:
folder = "cifar-10-batches-py"
preprocess_data(folder)
#valid_features, valid_labels = pickle.load(open('preprocess_validation.p','rb'))
#training_features,training_labels = pickle.load(open('preprocess_batch_1.p','rb'))
#test_features, test_labels = pickle.load(open('preprocess_test.p','rb'))

<h1>Hyperparameter</h1>

In [7]:
batch_size = 100
batch_num = 5
lr = 0.0001
epoch = 10
inp_size = 32
conv1 = {"size": 5, "nums": 32, "stride": 1}
conv2 = {"size": 5, "nums": 64, "stride": 1}
maxPool = {"size": 2, "stride": 2}
hidden_size1 = 1024
hidden_size2 = 512
out_size = 10

<h1>Placeholder</h1>

In [8]:
tf.reset_default_graph()
with tf.name_scope("data"):
    X = tf.placeholder(dtype=tf.float32, shape=[None, 32, 32, 3])
    y = tf.placeholder(dtype=tf.float32, shape=[None, 10])
    #training = tf.placeholder_with_default(False, shape=())

<h1>LeNet-5</h1>

In [9]:
def LeNet5(X):
    layer1_out = tf.layers.conv2d(X, conv1["nums"], conv1["size"], conv1["stride"], 'valid', activation=tf.nn.relu, kernel_initializer='he_normal')
    maxPool1_out = tf.contrib.layers.max_pool2d(layer1_out, maxPool["size"], maxPool["stride"], 'VALID')
    
    layer2_out = tf.layers.conv2d(maxPool1_out, conv2["nums"], conv2["size"], conv2["stride"], 'valid', activation=tf.nn.relu, kernel_initializer='he_normal')
    maxPool2_out = tf.contrib.layers.max_pool2d(layer2_out, maxPool["size"], maxPool["stride"], 'VALID')
    
    #layer2_drop = tf.layers.dropout(maxPool2_out, dropout_rate,  training=training)
    layer2_reshape = tf.reshape(maxPool2_out, shape=[-1, conv2["nums"] * 5 * 5])
    
    fc1 = tf.layers.dense(layer2_reshape, hidden_size1, activation=tf.nn.relu, use_bias=True, kernel_initializer='he_normal')
    #fc1_drop = tf.layers.dropout(fc1, dropout_rate,  training=training)
    fc2 =  tf.layers.dense(fc1, hidden_size2, activation=tf.nn.relu, use_bias=True, kernel_initializer='he_normal')
    
    out = tf.layers.dense(fc2, out_size, activation=tf.nn.softmax, use_bias=True, kernel_initializer='he_normal')
    return out
out = LeNet5(X)

<h1>Cost and Optimization</h1>

In [10]:
loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=out)
correct_pred = tf.equal(tf.argmax(out,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

optimizer = tf.train.AdamOptimizer(learning_rate=lr)#tf.train.MomentumOptimizer(lr, momentum=0.9, use_nesterov=True)
train_op = optimizer.minimize(loss)
init = tf.global_variables_initializer()

<h1>Training, validation and testing</h1>
<h2>Train your model only 10 epochs.</h2>
<h2>1.Print out validation accuracy after each training epoch</h2>
<h2>2.Print out training time for each training epoch</h2>
<h2>3.Print out testing accuracy</h2>

In [11]:
start = time.time()
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    for i in range(epoch):
        for j in range(1, batch_num + 1):
            file_name = 'preprocess_batch_' + str(j) + '.p'
            data = pickle.load(open(file_name, 'rb'))
            features, labels = data[0], data[1]
            tr_acc = 0
            count = 0
            for batch_x, batch_y in mini_batch(features, labels, batch_size):#load_preprocessed_training_batch(j, batch_size):
                sess.run(train_op, feed_dict={X:batch_x, y:batch_y})#, training:True})
                if j == batch_num: 
                    tr_acc += sess.run(accuracy,feed_dict={X:batch_x, y:batch_y})
                    count += 1
            if j == batch_num:
                print("epoch "+str(i)+", Training accuracy= {:.3f}".format(tr_acc / count))
        val_x, val_y = load_preprocessed_validation_batch()
        val_acc = sess.run(accuracy,feed_dict={X:val_x, y:val_y})
        print("epoch "+str(i)+", Validation accuracy= {:.3f}".format(val_acc))
        print("time elapsed at "+str(i)+"th epoch is: ", int(time.time() - start), " seconds")
    save_path = saver.save(sess, 'model.ckpt')
    print("Training finished!")
    acc_test = 0.0
    count = 0
    test_batch_size = 200
    for batch_x, batch_y in load_preprocessed_test_batch(test_batch_size):
            test_acc = sess.run(accuracy, feed_dict={X:batch_x, y:batch_y})
            acc_test += test_acc
            count += 1
    print("Test accuracy is: {:.3f}".format(acc_test / count))


epoch 0, Training accuracy= 0.372


ResourceExhaustedError: OOM when allocating tensor with shape[1000,32,14,14] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node MaxPool2D/MaxPool (defined at <ipython-input-9-b2c85c9ccffa>:3) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'MaxPool2D/MaxPool', defined at:
  File "/usr/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/us